<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Importing the key modules and set parameters

</div>

In [6]:
from meshparty import trimesh_io, trimesh_vtk
from caveclient import CAVEclient
import numpy as np
import pandas as pd
import cloudvolume
import itkwidgets


#setting up the cave client, if you have not done this before checkout the notebook CAVEsetup at 
#https://github.com/AllenInstitute/MicronsBinder/tree/master/notebooks/mm3_intro
client = CAVEclient()
#client.auth.get_new_token()
#client.auth.save_token(token="Your token here")


dataset_name = 'minnie65_public_v117'
client = CAVEclient(dataset_name)


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Using the CAVEclient, we can query the synapse table for any neuron in the volume
      </div>

In [2]:
client.materialize.get_tables()

['nucleus_detection_v0',
 'synapses_pni_2',
 'nucleus_neuron_svm',
 'proofreading_status_public_release',
 'func_unit_em_match_release',
 'allen_soma_ei_class_model_v1',
 'allen_visp_column_soma_coarse_types_v1']

In [3]:
#selecting for only the proofread cells  
proofread = client.materialize.query_table('proofreading_status_public_release')
proofread.head()


,id,valid,pt_supervoxel_id,pt_root_id,valid_id,status_dendrite,status_axon,pt_position
0,1,t,89529934389098311,864691136296964635,864691136296964635,extended,non,"[179808, 216672, 23361]"
1,2,t,90584228533843146,864691136311986237,864691136311986237,extended,non,"[187840, 207232, 22680]"
2,3,t,89528353773943370,864691135355207119,864691135355207119,extended,non,"[180016, 204592, 22798]"
3,4,t,91077153340676495,864691135355207375,864691135355207375,extended,non,"[191424, 209888, 22845]"
4,5,t,88897234233461709,864691136422983727,864691136422983727,extended,non,"[175248, 220944, 23561]"


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> We can go on to find the post_synaptic cell id (post_pt_root_id) with the most synapses. (Hint, we recommend focusing on the proofread cells)
        </div>

In [21]:
#synapse table with all the synapses in the volume (proofread and non-proofread cells)
all_syn_df = client.materialize.query_table('synapses_pni_2')

#subselecting only synapses onto proofread cells
proofread_synapses = all_syn_df[all_syn_df.post_pt_root_id.isin(proofread.pt_root_id)]
proofread_synapses['syn_num']=proofread_synapses.groupby('post_pt_root_id')['id'].transform(len)

#which proofread cell id has the largest number of post-synaptic synapses
most_syn = max(proofread_synapses.syn_num) 

#since there may be multiple cells with the same maximum number, we will just take the first one
cellid = proofread_synapses[proofread_synapses.syn_num==most_syn]['post_pt_root_id'].values[0] 


#segmentation ID for the proofread cell with the greatest number of synapses
print('Cell ID: %d, Number of synapses: %d'%(cellid,most_syn))


Cell ID: 864691135837451155, Number of synapses: 16



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Now we can move on to visualize this cell in 3D with some of it's biggest synapses
        </div>

In [22]:
seg_source = client.info.segmentation_source()

#using the google cloud segmentation path
cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)

#you can also try the aws segmentation path
#cv = cloudvolume.CloudVolume("precomputed://s3://bossdb-open-data/microns/minnie/minnie65-flat-seg/", use_https=True)

mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])



In [23]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [33]:
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))
biggest_synapses = post_synapse_df.sort_values(by=['size'],ascending=False).head(10)
print(biggest_synapses[['pre_pt_root_id', 'size']])


           pre_pt_root_id   size
16968  864691135347316908  92392
20433  864691136100692752  79068
20062  864691135228996162  76772
11897  864691133644557380  75988
21347  864691135360920908  73300
1817   864691135296375944  72376
2338   864691134978346775  71860
19604  864691136705563246  71648
5292   864691136337720348  71572
24604  864691134929711911  71568


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Lastly we will visualize the cell and these 10 synapses, please note that you need to increase the size of the synapses using the slider in the widget tool
        </div>

In [31]:
#mm = trimesh_io.MeshMeta(disk_cache_path='test/test_files')
#mesh = mm.mesh(filename ='/data/dynamic_brain_workshop/electron_microscopy/2021/meshes/%d.h5'%cellid)
mesh_poly =trimesh_vtk.trimesh_to_vtk(mesh.vertices,mesh.faces,None)

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(biggest_synapses['ctr_pt_position'].values) * voxel_resolution
syn_sizes = biggest_synapses['size']
syn_actors = trimesh_vtk.point_cloud_actor(syn_pts, size=syn_sizes.values)
viewer2 =itkwidgets.view(geometries=[poly_data],
                        point_sets = [syn_pts],
                        point_sets_colors = ['b'])



In [32]:
#when visualizing the synapses, they will initially appear very small, be sure to increase the size of the 
#synapses using the sliding bar under Point Set 0. You can also change the color using color map in the widget
#You should be able to see a pyramidal neuron with 10 points indicating where it receives its 10 largest synapses
viewer2

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…